In [1]:
!ipconfig


Windows IP Configuration


Ethernet adapter Ethernet:

   Media State . . . . . . . . . . . : Media disconnected
   Connection-specific DNS Suffix  . : fc.up.pt

Ethernet adapter Ethernet 2:

   Connection-specific DNS Suffix  . : 
   Link-local IPv6 Address . . . . . : fe80::9627:98d9:5bff:e9bd%3
   IPv4 Address. . . . . . . . . . . : 192.168.56.1
   Subnet Mask . . . . . . . . . . . : 255.255.255.0
   Default Gateway . . . . . . . . . : 

Ethernet adapter Ethernet 3:

   Media State . . . . . . . . . . . : Media disconnected
   Connection-specific DNS Suffix  . : 

Wireless LAN adapter Liga��o de �rea Local* 1:

   Media State . . . . . . . . . . . : Media disconnected
   Connection-specific DNS Suffix  . : 

Wireless LAN adapter Liga��o de �rea Local* 2:

   Media State . . . . . . . . . . . : Media disconnected
   Connection-specific DNS Suffix  . : 

Wireless LAN adapter Wi-Fi:

   Connection-specific DNS Suffix  . : fc.up.pt
   Link-local IPv6 Address . . . . . : fe80::5276:1a0c

In [1]:
######Funcoes para o fit automatico######
#### na prática vamos usar a função mais abaixo - iterative_find_minimum_gaussian(wl, sp, range_min, range_max, delta)

from scipy.optimize import curve_fit

def gaussian(x, a, b, c):
    return a * np.exp(-0.5 * ((x - b) / c)**2)


def find_minimum_gaussian(wl, sp, range_min=1550, range_max=1600):
    # Filter data within the specified range
    mask = (wl >= range_min) & (wl <= range_max)
    x_data = wl[mask]
    y_data = sp[mask]

    if len(x_data) == 0:
        return None, None

    # Initial guesses for Gaussian parameters:
    # a: amplitude (estimated from y_data min),
    # b: mean (estimated from x_data corresponding to y_data min),
    # c: std dev (rough guess based on the range width)
    initial_guess = [min(y_data), x_data[np.argmin(y_data)], (range_max - range_min) / 6]
    
    try:
        # Fit the Gaussian model
        params, _ = curve_fit(gaussian, x_data, y_data, p0=initial_guess)
        # Compute the minimum of the Gaussian fit
        a, b, c = params
        gaussian_min = gaussian(b, *params)  # Gaussian minimum at the center b
        return b, gaussian_min, x_data, gaussian(x_data, *params)
    except Exception as e:
        print(f"Fit error: {e}")
        return None, None
    

#####################################3

def iterative_find_minimum_gaussian(wl, sp, range_min=1550, range_max=1600, delta=5):
    #encontra 2 vezes consecutivas o mínimo, fazendo o improvement da estimativa
    min_wavelength, min_intensity, xregion, yfit = find_minimum_gaussian(wl, sp, range_min=range_min, range_max=range_max)
    return find_minimum_gaussian(wl, sp, range_min=min_wavelength-delta, range_max=min_wavelength+delta)


def iterative_find_minimum_gaussian(wl, sp, range_min=1550, range_max=1600, delta=5,delta2=5):
    #encontra 2 vezes consecutivas o mínimo, fazendo o improvement da estimativa
    min_wavelength, min_intensity, xregion, yfit = find_minimum_gaussian(wl, sp, range_min=range_min, range_max=range_max)
    return find_minimum_gaussian(wl, sp, range_min=min_wavelength-delta, range_max=min_wavelength+delta)
    


import pandas as pd

def read_spectral_data(file_path):
    # Read the Excel file
    timestamp = file_path.split('_')[-1].split('.')[0]
    df = pd.read_csv(file_path,sep="\t")
    
    # Assume the first column is 'Wavelength' and the others are 'Channel 1', 'Channel 2', etc.
    wavelength = df['Wavelength (nm)'].values
    channels = [df[column].values for column in df.columns if 'Spectra' in column]
    
    # Return the wavelength array and the channel arrays
    return wavelength, channels[2],channels[3], timestamp

In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import dash_daq as daq
import os
from datetime import datetime
import random
import time

# Path to the folder containing sensor data files
sensor_data_folder = 'C:\\Users\\nunoa\\Academia de Física 2024\\dados_interrogacao\\Aquisicao_1'

# Global variable to hold sensor data
sensor_data = pd.DataFrame(columns=['Time', 'Sensor Value'])

# Function to get the most recent sensor data file
def get_most_recent_file(folder):
    files = [os.path.join(folder, f) for f in os.listdir(folder) if f.startswith('Spectra') and f.endswith('.xls')]
    if not files:
        return None
    most_recent_file = max(files, key=os.path.getmtime)
    return most_recent_file

# Function to read sensor data from the most recent file
def read_sensor_data():
    most_recent_file = get_most_recent_file(sensor_data_folder)
    if most_recent_file:
        
        wavelength, data_lpg1, data_lpg2, timestamp = read_spectral_data(most_recent_file)
        min_wavelength_LPG1, min_intensity, xregion, yfit = iterative_find_minimum_gaussian(wavelength, data_lpg1,range_min = 1520, range_max=1600,delta=2)
        sensor_value = min_wavelength_LPG1  # Simulated sensor value
        current_time = datetime.now()  # Current date and time
        return current_time, sensor_value
    else:
        return None, None
    
def read_sensor_data2():
    most_recent_file = get_most_recent_file(sensor_data_folder)
    if most_recent_file:
        
        wavelength, data_lpg1, data_lpg2, timestamp = read_spectral_data(most_recent_file)
        min_wavelength_LPG1, min_intensity, xregion, yfit = iterative_find_minimum_gaussian(wavelength, data_lpg2,range_min = 1520, range_max=1600,delta=2)
        sensor_value = min_wavelength_LPG1  # Simulated sensor value
        current_time = datetime.now()  # Current date and time
        return current_time, sensor_value
    else:
        return None, None
    
read_sensor_data()

(datetime.datetime(2024, 5, 17, 19, 3, 39, 114567), 1557.0011423204448)

In [4]:
import dash
from dash import html, dcc
import dash_daq as daq
from dash.dependencies import Output, Input
import plotly.graph_objs as go
import pandas as pd

# Create Dash application
app = dash.Dash(__name__)

# Initialize global variables for sensor data
sensor_data_1 = pd.DataFrame(columns=['Time', 'Sensor Value'])
sensor_data_2 = pd.DataFrame(columns=['Time', 'Sensor Value'])

# Placeholder functions to read sensor data
def read_sensor_data_1():
    # Replace with actual data reading logic
    return pd.Timestamp.now(), 1500  # Example data

def read_sensor_data_2():
    # Replace with actual data reading logic
    return pd.Timestamp.now(), 1600  # Example data

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Sensor Data Dashboard", style={'textAlign': 'center', 'color': '#FFFFFF', 'backgroundColor': '#333333'}),
    html.Div([
        html.Div([
            dcc.Graph(id='sensor-graph-1')
        ], style={'width': '60%', 'display': 'inline-block'}),
        html.Div([
            html.Div(id='water-quality-1', style={'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': '#FF4136', 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}),
            daq.Gauge(
                id='water-quality-gauge-1',
                min=0,
                max=10,
                value=5,
                color={"gradient": True, "ranges": {"green": [0, 4], "yellow": [4, 7], "red": [7, 10]}},
                label='Water Quality 1',
                style={'margin': '20px auto'}
            )
        ], style={'width': '35%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ]),
    html.Div([
        html.Div([
            dcc.Graph(id='sensor-graph-2')
        ], style={'width': '60%', 'display': 'inline-block'}),
        html.Div([
            html.Div(id='water-quality-2', style={'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': '#FF4136', 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}),
            daq.Gauge(
                id='water-quality-gauge-2',
                min=0,
                max=10,
                value=5,
                color={"gradient": True, "ranges": {"green": [0, 4], "yellow": [4, 7], "red": [7, 10]}},
                label='Water Quality 2',
                style={'margin': '20px auto'}
            )
        ], style={'width': '35%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ]),
    dcc.Interval(
        id='interval-component',
        interval=3*1000,  # Update every 3 seconds
        n_intervals=0
    )
], style={'backgroundColor': '#2F2F2F', 'padding': '20px'})

# Define callback to update graph, water quality status, and gauge for Sensor 1
@app.callback(
    [Output('sensor-graph-1', 'figure'),
     Output('water-quality-1', 'children'),
     Output('water-quality-gauge-1', 'value'),
     Output('water-quality-1', 'style')],
    [Input('interval-component', 'n_intervals')]
)
def update_graph_live_1(n):
    global sensor_data_1

    # Read new sensor data for Sensor 1
    new_time, new_value = read_sensor_data()
    
    if new_time and new_value:
        new_row = pd.DataFrame({'Time': [new_time], 'Sensor Value': [new_value]})
        sensor_data_1 = pd.concat([sensor_data_1, new_row], ignore_index=True)

    # Determine water quality based on the last sensor value for Sensor 1
    latest_value = sensor_data_1['Sensor Value'].iloc[-1] if not sensor_data_1.empty else 0
    if latest_value > 1557.5:  # Example threshold value
        water_quality = "Water Quality: Bad"
        gauge_value = 8
        color = '#FF4136'
    else:
        water_quality = "Water Quality: Good"
        gauge_value = 2
        color = '#2ECC40'

    figure = {
        'data': [
            go.Scatter(
                x=sensor_data_1['Time'],
                y=sensor_data_1['Sensor Value'],
                mode='lines+markers',
                marker=dict(color='#00CC96')
            )
        ],
        'layout': go.Layout(
            title='Real-time Sensor Data 1',
            xaxis={'title': 'Time', 'gridcolor': '#4D4D4D'},
            yaxis={'title': 'Sensor Value', 'gridcolor': '#4D4D4D'},
            paper_bgcolor='#2F2F2F',
            plot_bgcolor='#2F2F2F',
            font={'color': '#FFFFFF'}
        )
    }
    text_style = {'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': color, 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}
    return figure, water_quality, gauge_value, text_style

# Define callback to update graph, water quality status, and gauge for Sensor 2
@app.callback(
    [Output('sensor-graph-2', 'figure'),
     Output('water-quality-2', 'children'),
     Output('water-quality-gauge-2', 'value'),
     Output('water-quality-2', 'style')],
    [Input('interval-component', 'n_intervals')]
)
def update_graph_live_2(n):
    global sensor_data_2

    # Read new sensor data for Sensor 2
    new_time, new_value = read_sensor_data()
    
    if new_time and new_value:
        new_row = pd.DataFrame({'Time': [new_time], 'Sensor Value': [new_value]})
        sensor_data_2 = pd.concat([sensor_data_2, new_row], ignore_index=True)

    # Determine water quality based on the last sensor value for Sensor 2
    latest_value = sensor_data_2['Sensor Value'].iloc[-1] if not sensor_data_2.empty else 0
    if latest_value > 1557.5:  # Example threshold value
        water_quality = "Water Quality: Bad"
        gauge_value = 8
        color = '#FF4136'
    else:
        water_quality = "Water Quality: Good"
        gauge_value = 2
        color = '#2ECC40'

    figure = {
        'data': [
            go.Scatter(
                x=sensor_data_2['Time'],
                y=sensor_data_2['Sensor Value'],
                mode='lines+markers',
                marker=dict(color='#00CC96')
            )
        ],
        'layout': go.Layout(
            title='Real-time Sensor Data 2',
            xaxis={'title': 'Time', 'gridcolor': '#4D4D4D'},
            yaxis={'title': 'Sensor Value', 'gridcolor': '#4D4D4D'},
            paper_bgcolor='#2F2F2F',
            plot_bgcolor='#2F2F2F',
            font={'color': '#FFFFFF'})
    }
    text_style = {'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': color, 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}
    return figure, water_quality, gauge_value, text_style

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True, host='172.17.1.247', port=8050)  # Accessible from any IP



In [ ]:
# Create Dash application
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Sensor Data Dashboard", style={'textAlign': 'center', 'color': '#FFFFFF', 'backgroundColor': '#333333'}),
    html.Div([
        html.Div([
            dcc.Graph(id='sensor-graph-1')
        ], style={'width': '60%', 'display': 'inline-block'}),
        html.Div([
            html.Div(id='water-quality-1', style={'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': '#FF4136', 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}),
            daq.Gauge(
                id='water-quality-gauge-1',
                min=0,
                max=10,
                value=5,
                color={"gradient": True, "ranges": {"green": [0, 4], "yellow": [4, 7], "red": [7, 10]}},
                label='Water Quality 1',
                style={'margin': '20px auto'}
            )
        ], style={'width': '35%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ]),
    html.Div([
        html.Div([
            dcc.Graph(id='sensor-graph-2')
        ], style={'width': '60%', 'display': 'inline-block'}),
        html.Div([
            html.Div(id='water-quality-2', style={'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': '#FF4136', 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}),
            daq.Gauge(
                id='water-quality-gauge-2',
                min=0,
                max=10,
                value=5,
                color={"gradient": True, "ranges": {"green": [0, 4], "yellow": [4, 7], "red": [7, 10]}},
                label='Water Quality 2',
                style={'margin': '20px auto'}
            )
        ], style={'width': '35%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ]),
    dcc.Interval(
        id='interval-component',
        interval=3*1000,  # Update every 3 seconds
        n_intervals=0
    )
], style={'backgroundColor': '#2F2F2F', 'padding': '20px'})

# Define callback to update graph, water quality status, and gauge for Sensor 1
@app.callback(
    [Output('sensor-graph-1', 'figure'),
     Output('water-quality-1', 'children'),
     Output('water-quality-gauge-1', 'value'),
     Output('water-quality-1', 'style')],
    [Input('interval-component', 'n_intervals')]
)
def update_graph_live_1(n):
    global sensor_data_1

    # Read new sensor data for Sensor 1
    new_time, new_value = read_sensor_data_1()
    
    if new_time and new_value:
        new_row = pd.DataFrame({'Time': [new_time], 'Sensor Value': [new_value]})
        sensor_data_1 = pd.concat([sensor_data_1, new_row], ignore_index=True)

    # Determine water quality based on the last sensor value for Sensor 1
    latest_value = sensor_data_1['Sensor Value'].iloc[-1] if not sensor_data_1.empty else 0
    if latest_value > 1557.5:  # Example threshold value
        water_quality = "Water Quality: Bad"
        gauge_value = 8
        color = '#FF4136'
    else:
        water_quality = "Water Quality: Good"
        gauge_value = 2
        color = '#2ECC40'

    figure = {
        'data': [
            go.Scatter(
                x=sensor_data_1['Time'],
                y=sensor_data_1['Sensor Value'],
                mode='lines+markers',
                marker=dict(color='#00CC96')
            )
        ],
        'layout': go.Layout(
            title='Real-time Sensor Data 1',
            xaxis={'title': 'Time', 'gridcolor': '#4D4D4D'},
            yaxis={'title': 'Sensor Value', 'gridcolor': '#4D4D4D'},
            paper_bgcolor='#2F2F2F',
            plot_bgcolor='#2F2F2F',
            font={'color': '#FFFFFF'}
        )
    }
    text_style = {'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': color, 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}
    return figure, water_quality, gauge_value, text_style

# Define callback to update graph, water quality status, and gauge for Sensor 2
@app.callback(
    [Output('sensor-graph-2', 'figure'),
     Output('water-quality-2', 'children'),
     Output('water-quality-gauge-2', 'value'),
     Output('water-quality-2', 'style')],
    [Input('interval-component', 'n_intervals')]
)
def update_graph_live_2(n):
    global sensor_data_2

    # Read new sensor data for Sensor 2
    new_time, new_value = read_sensor_data_2()
    
    if new_time and new_value:
        new_row = pd.DataFrame({'Time': [new_time], 'Sensor Value': [new_value]})
        sensor_data_2 = pd.concat([sensor_data_2, new_row], ignore_index=True)

    # Determine water quality based on the last sensor value for Sensor 2
    latest_value = sensor_data_2['Sensor Value'].iloc[-1] if not sensor_data_2.empty else 0
    if latest_value > 1557.5:  # Example threshold value
        water_quality = "Water Quality: Bad"
        gauge_value = 8
        color = '#FF4136'
    else:
        water_quality = "Water Quality: Good"
        gauge_value = 2
        color = '#2ECC40'

    figure = {
        'data': [
            go.Scatter(
                x=sensor_data_2['Time'],
                y=sensor_data_2['Sensor Value'],
                mode='lines+markers',
                marker=dict(color='#00CC96')
            )
        ],
        'layout': go.Layout(
            title='Real-time Sensor Data 2',
            xaxis={'title': 'Time', 'gridcolor': '#4D4D4D'},
            yaxis={'title': 'Sensor Value', 'gridcolor': '#4D4D4D'},
            paper_bgcolor='#2F2F2F',
            plot_bgcolor='#2F2F2F',
            font={'color': '#FFFFFF'}
        )
    }
    text_style = {'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': color, 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}
    return figure, water_quality, gauge_value, text_style

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True, host='172.17.1.247', port=8050)  # Accessible from any IP


In [ ]:
# Create Dash application
app = dash.Dash(__name__)



# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Sensor Data Dashboard", style={'textAlign': 'center', 'color': '#FFFFFF', 'backgroundColor': '#333333'}),
    html.Div([
        html.Div([
            dcc.Graph(id='sensor-graph')
        ], style={'width': '60%', 'display': 'inline-block'}),
        html.Div([
            html.Div(id='water-quality', style={'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': '#FF4136', 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}),
            daq.Gauge(
                id='water-quality-gauge',
                min=0,
                max=10,
                value=5,
                color={"gradient":True,"ranges":{"green":[0,4],"yellow":[4,7],"red":[7,10]}},
                label='Water Quality',
                style={'margin': '20px auto'}
            )
        ], style={'width': '35%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ]),
    dcc.Interval(
        id='interval-component',
        interval=3*1000,  # Update every 3 seconds
        n_intervals=0
    )
], style={'backgroundColor': '#2F2F2F', 'padding': '20px'})

# Define callback to update graph, water quality status, and gauge
@app.callback(
    [Output('sensor-graph', 'figure'),
     Output('water-quality', 'children'),
     Output('water-quality-gauge', 'value'),
     Output('water-quality', 'style')],
    [Input('interval-component', 'n_intervals')]
)
def update_graph_live(n):
    global sensor_data

    # Read new sensor data
    new_time, new_value = read_sensor_data()
    
    if new_time and new_value:
        new_row = pd.DataFrame({'Time': [new_time], 'Sensor Value': [new_value]})
        sensor_data = pd.concat([sensor_data, new_row], ignore_index=True)

    # Determine water quality based on the last sensor value
    latest_value = sensor_data['Sensor Value'].iloc[-1] if not sensor_data.empty else 0
    if latest_value > 1557.5:  # Example threshold value
        water_quality = "Water Quality: Bad"
        gauge_value = 8
        color = '#FF4136'
    else:
        water_quality = "Water Quality: Good"
        gauge_value = 2
        color = '#2ECC40'

    figure = {
        'data': [
            go.Scatter(
                x=sensor_data['Time'],
                y=sensor_data['Sensor Value'],
                mode='lines+markers',
                marker=dict(color='#00CC96')
            )
        ],
        'layout': go.Layout(
            title='Real-time Sensor Data',
            xaxis={'title': 'Time', 'gridcolor': '#4D4D4D'},
            yaxis={'title': 'Sensor Value', 'gridcolor': '#4D4D4D'},
            paper_bgcolor='#2F2F2F',
            plot_bgcolor='#2F2F2F',
            font={'color': '#FFFFFF'}
        )
    }
    text_style = {'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': color, 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}
    return figure, water_quality, gauge_value, text_style

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True, host='172.17.1.247', port=8050)  # Accessible from any IP


In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import dash_daq as daq

# Create Dash application
app = dash.Dash(__name__)

# Generate some example sensor data
def generate_sensor_data():
    time_series = pd.date_range(start='2023-01-01', periods=100, freq='T')
    sensor_values = np.cumsum(np.random.randn(100))  # Random walk data
    data = pd.DataFrame({'Time': time_series, 'Sensor Value': sensor_values})
    return data

sensor_data = generate_sensor_data()

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Sensor Data Dashboard", style={'textAlign': 'center', 'color': '#FFFFFF', 'backgroundColor': '#333333'}),
    html.Div([
        html.Div([
            dcc.Graph(id='sensor-graph')
        ], style={'width': '60%', 'display': 'inline-block'}),
        html.Div([
            html.Div(id='water-quality', style={'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': '#FF4136', 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}),
            daq.Gauge(
                id='water-quality-gauge',
                min=0,
                max=10,
                value=5,
                color={"gradient":True,"ranges":{"green":[0,4],"yellow":[4,7],"red":[7,10]}},
                label='Water Quality',
                style={'margin': '20px auto'}
            )
        ], style={'width': '35%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ]),
    dcc.Interval(
        id='interval-component',
        interval=1*1000,  # Update every 1 second
        n_intervals=0
    )
], style={'backgroundColor': '#2F2F2F', 'padding': '20px'})

# Define callback to update graph, water quality status, and gauge
@app.callback(
    [Output('sensor-graph', 'figure'),
     Output('water-quality', 'children'),
     Output('water-quality-gauge', 'value'),
     Output('water-quality', 'style')],
    [Input('interval-component', 'n_intervals')]
)
def update_graph_live(n):
    # Simulate new data for the example
    sensor_data = generate_sensor_data()

    # Determine water quality based on the last sensor value
    latest_value = sensor_data['Sensor Value'].iloc[-1]
    if latest_value > 1:  # Example threshold value
        water_quality = "Water Quality: Bad"
        gauge_value = 8
        color = '#FF4136'
    else:
        water_quality = "Water Quality: Good"
        gauge_value = 2
        color = '#2ECC40'

    figure = {
        'data': [
            go.Scatter(
                x=sensor_data['Time'],
                y=sensor_data['Sensor Value'],
                mode='lines+markers',
                marker=dict(color='#00CC96')
            )
        ],
        'layout': go.Layout(
            title='Real-time Sensor Data',
            xaxis={'title': 'Time', 'gridcolor': '#4D4D4D'},
            yaxis={'title': 'Sensor Value', 'gridcolor': '#4D4D4D'},
            paper_bgcolor='#2F2F2F',
            plot_bgcolor='#2F2F2F',
            font={'color': '#FFFFFF'}
        )
    }
    text_style = {'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': color, 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}
    return figure, water_quality, gauge_value, text_style

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True, host='192.168.1.66',port=8050)  # Accessible from any IP

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import dash_daq as daq
import os
from datetime import datetime
import random

# Create Dash application
app = dash.Dash(__name__)

# Path to the folder containing sensor data files
sensor_data_folder = 'C:\\Users\\nunoa\\Academia de Física 2024\\dados_interrogacao\\Aquisicao_1\\'

# Function to get the most recent sensor data file
def get_most_recent_file(folder):
    files = [os.path.join(folder, f) for f in os.listdir(folder) if f.startswith('Spectra') and f.endswith('.xls')]
    if not files:
        return None
    most_recent_file = max(files, key=os.path.getmtime)
    return most_recent_file

# Function to generate sensor data
def generate_sensor_data():
    most_recent_file = get_most_recent_file(sensor_data_folder)
    if most_recent_file:
        # For now, we output a random value
        sensor_value = random.uniform(-2, 2)
        file_time_str = os.path.basename(most_recent_file).split('-')[1:6]
        file_time_str = '-'.join(file_time_str)
        file_time = datetime.strptime(file_time_str, '%d-%m-%Y_%H-%M-%S')
        time_series = pd.date_range(start=file_time, periods=100, freq='T')
        sensor_values = np.cumsum(np.random.randn(100)) + sensor_value  # Random walk data
        data = pd.DataFrame({'Time': time_series, 'Sensor Value': sensor_values})
        return data
    else:
        return pd.DataFrame(columns=['Time', 'Sensor Value'])

# Initial data load
sensor_data = generate_sensor_data()

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Sensor Data Dashboard", style={'textAlign': 'center', 'color': '#FFFFFF', 'backgroundColor': '#333333'}),
    html.Div([
        html.Div([
            dcc.Graph(id='sensor-graph')
        ], style={'width': '60%', 'display': 'inline-block'}),
        html.Div([
            html.Div(id='water-quality', style={'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': '#FF4136', 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}),
            daq.Gauge(
                id='water-quality-gauge',
                min=0,
                max=10,
                value=5,
                color={"gradient":True,"ranges":{"green":[0,4],"yellow":[4,7],"red":[7,10]}},
                label='Water Quality',
                style={'margin': '20px auto'}
            )
        ], style={'width': '35%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ]),
    dcc.Interval(
        id='interval-component',
        interval=1*1000,  # Update every 1 second
        n_intervals=0
    )
], style={'backgroundColor': '#2F2F2F', 'padding': '20px'})

# Define callback to update graph, water quality status, and gauge
@app.callback(
    [Output('sensor-graph', 'figure'),
     Output('water-quality', 'children'),
     Output('water-quality-gauge', 'value'),
     Output('water-quality', 'style')],
    [Input('interval-component', 'n_intervals')]
)
def update_graph_live(n):
    # Simulate new data for the example
    sensor_data = generate_sensor_data()

    # Determine water quality based on the last sensor value
    latest_value = sensor_data['Sensor Value'].iloc[-1] if not sensor_data.empty else 0
    if latest_value > 1:  # Example threshold value
        water_quality = "Water Quality: Bad"
        gauge_value = 8
        color = '#FF4136'
    else:
        water_quality = "Water Quality: Good"
        gauge_value = 2
        color = '#2ECC40'

    figure = {
        'data': [
            go.Scatter(
                x=sensor_data['Time'],
                y=sensor_data['Sensor Value'],
                mode='lines+markers',
                marker=dict(color='#00CC96')
            )
        ],
        'layout': go.Layout(
            title='Real-time Sensor Data',
            xaxis={'title': 'Time', 'gridcolor': '#4D4D4D'},
            yaxis={'title': 'Sensor Value', 'gridcolor': '#4D4D4D'},
            paper_bgcolor='#2F2F2F',
            plot_bgcolor='#2F2F2F',
            font={'color': '#FFFFFF'}
        )
    }
    text_style = {'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': color, 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}
    return figure, water_quality, gauge_value, text_style

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True, host='192.168.1.66', port=8050)  # Accessible from any IP


In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import dash_daq as daq
import os
from datetime import datetime
import random

# Create Dash application
app = dash.Dash(__name__)

# Path to the folder containing sensor data files
sensor_data_folder = 'C:\\Users\\nunoa\\Academia de Física 2024\\dados_interrogacao\\Aquisicao_1\\'

# Function to get the most recent sensor data file
def get_most_recent_file(folder):
    files = [os.path.join(folder, f) for f in os.listdir(folder) if f.startswith('Spectra') and f.endswith('.xls')]
    if not files:
        return None
    most_recent_file = max(files, key=os.path.getmtime)
    return most_recent_file

# Function to read sensor data from the most recent file
def read_sensor_data():
    most_recent_file = get_most_recent_file(sensor_data_folder)
    if most_recent_file:
        sensor_value = random.uniform(-2, 2)  # Simulated sensor value
        current_time = datetime.now()  # Current date and time
        return current_time, sensor_value
    else:
        return None, None
# Initial data load
sensor_data = read_sensor_data()

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Sensor Data Dashboard", style={'textAlign': 'center', 'color': '#FFFFFF', 'backgroundColor': '#333333'}),
    html.Div([
        html.Div([
            dcc.Graph(id='sensor-graph')
        ], style={'width': '60%', 'display': 'inline-block'}),
        html.Div([
            html.Div(id='water-quality', style={'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': '#FF4136', 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}),
            daq.Gauge(
                id='water-quality-gauge',
                min=0,
                max=10,
                value=5,
                color={"gradient":True,"ranges":{"green":[0,4],"yellow":[4,7],"red":[7,10]}},
                label='Water Quality',
                style={'margin': '20px auto'}
            )
        ], style={'width': '35%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ]),
    dcc.Interval(
        id='interval-component',
        interval=1*1000,  # Update every 1 second
        n_intervals=0
    )
], style={'backgroundColor': '#2F2F2F', 'padding': '20px'})

# Define callback to update graph, water quality status, and gauge
@app.callback(
    [Output('sensor-graph', 'figure'),
     Output('water-quality', 'children'),
     Output('water-quality-gauge', 'value'),
     Output('water-quality', 'style')],
    [Input('interval-component', 'n_intervals')]
)
def update_graph_live(n):
    # Simulate new data for the example
    sensor_data = read_sensor_data()
    
    # Determine water quality based on the last sensor value
    latest_value = sensor_data['Sensor Value'].iloc[-1] if not sensor_data.empty else 0
    if latest_value > 1:  # Example threshold value
        water_quality = "Water Quality: Bad"
        gauge_value = 8
        color = '#FF4136'
    else:
        water_quality = "Water Quality: Good"
        gauge_value = 2
        color = '#2ECC40'

    figure = {
        'data': [
            go.Scatter(
                x=sensor_data['Time'],
                y=sensor_data['Sensor Value'],
                mode='lines+markers',
                marker=dict(color='#00CC96')
            )
        ],
        'layout': go.Layout(
            title='Real-time Sensor Data',
            xaxis={'title': 'Time', 'gridcolor': '#4D4D4D'},
            yaxis={'title': 'Sensor Value', 'gridcolor': '#4D4D4D'},
            paper_bgcolor='#2F2F2F',
            plot_bgcolor='#2F2F2F',
            font={'color': '#FFFFFF'}
        )
    }
    text_style = {'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': color, 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}
    return figure, water_quality, gauge_value, text_style

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True, host='192.168.1.66', port=8050)  # Accessible from any IP


In [ ]:
######Funcoes para o fit automatico######
#### na prática vamos usar a função mais abaixo - iterative_find_minimum_gaussian(wl, sp, range_min, range_max, delta)

from scipy.optimize import curve_fit

def gaussian(x, a, b, c):
    return a * np.exp(-0.5 * ((x - b) / c)**2)


def find_minimum_gaussian(wl, sp, range_min=1550, range_max=1600):
    # Filter data within the specified range
    mask = (wl >= range_min) & (wl <= range_max)
    x_data = wl[mask]
    y_data = sp[mask]

    if len(x_data) == 0:
        return None, None

    # Initial guesses for Gaussian parameters:
    # a: amplitude (estimated from y_data min),
    # b: mean (estimated from x_data corresponding to y_data min),
    # c: std dev (rough guess based on the range width)
    initial_guess = [min(y_data), x_data[np.argmin(y_data)], (range_max - range_min) / 6]
    
    try:
        # Fit the Gaussian model
        params, _ = curve_fit(gaussian, x_data, y_data, p0=initial_guess)
        # Compute the minimum of the Gaussian fit
        a, b, c = params
        gaussian_min = gaussian(b, *params)  # Gaussian minimum at the center b
        return b, gaussian_min, x_data, gaussian(x_data, *params)
    except Exception as e:
        print(f"Fit error: {e}")
        return None, None
    

#####################################3

def iterative_find_minimum_gaussian(wl, sp, range_min=1550, range_max=1600, delta=5):
    #encontra 2 vezes consecutivas o mínimo, fazendo o improvement da estimativa
    min_wavelength, min_intensity, xregion, yfit = find_minimum_gaussian(wl, sp, range_min=range_min, range_max=range_max)
    return find_minimum_gaussian(wl, sp, range_min=min_wavelength-delta, range_max=min_wavelength+delta)


def iterative_find_minimum_gaussian(wl, sp, range_min=1550, range_max=1600, delta=5,delta2=5):
    #encontra 2 vezes consecutivas o mínimo, fazendo o improvement da estimativa
    min_wavelength, min_intensity, xregion, yfit = find_minimum_gaussian(wl, sp, range_min=range_min, range_max=range_max)
    return find_minimum_gaussian(wl, sp, range_min=min_wavelength-delta, range_max=min_wavelength+delta)
    

import datetime
from datetime import datetime
import matplotlib.dates
import pandas as pd

def read_spectral_data(file_path):
    # Read the Excel file
    timestamp = file_path.split('_')[-1].split('.')[0]
    df = pd.read_csv(file_path,sep="\t")
    
    # Assume the first column is 'Wavelength' and the others are 'Channel 1', 'Channel 2', etc.
    wavelength = df['Wavelength (nm)'].values
    channels = [df[column].values for column in df.columns if 'Spectra' in column]
    
    # Return the wavelength array and the channel arrays
    return wavelength, channels[2],channels[3], timestamp

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import dash_daq as daq
import os
from datetime import datetime
import random
import time

# Create Dash application
app = dash.Dash(__name__)

# Path to the folder containing sensor data files
sensor_data_folder = 'C:\\Users\\nunoa\\Academia de Física 2024\\dados_interrogacao\\Aquisicao_1\\'

# Global variable to hold sensor data
sensor_data = pd.DataFrame(columns=['Time', 'Sensor Value'])

# Function to get the most recent sensor data file
def get_most_recent_file(folder):
    files = [os.path.join(folder, f) for f in os.listdir(folder) if f.startswith('Spectra') and f.endswith('.xls')]
    if not files:
        return None
    most_recent_file = max(files, key=os.path.getmtime)
    print(most_recent_file)
    return most_recent_file

# Function to read sensor data from the most recent file
def read_sensor_data():
    most_recent_file = get_most_recent_file(sensor_data_folder)
    if most_recent_file:
        sensor_value = random.uniform(-2, 2)  # Simulated sensor value
        #wavelength, data_lpg1, data_lpg2, timestamp = read_spectral_data(most_recent_file)
        #min_wavelength_LPG1, min_intensity, xregion, yfit = iterative_find_minimum_gaussian(wavelength, data_lpg1,range_min = 1520, range_max=1600,delta=2)
        current_time = datetime.now()  # Current date and time
        return current_time, sensor_value
    else:
        return None, None

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Sensor Data Dashboard", style={'textAlign': 'center', 'color': '#FFFFFF', 'backgroundColor': '#333333'}),
    html.Div([
        html.Div([
            dcc.Graph(id='sensor-graph')
        ], style={'width': '60%', 'display': 'inline-block'}),
        html.Div([
            html.Div(id='water-quality', style={'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': '#FF4136', 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}),
            daq.Gauge(
                id='water-quality-gauge',
                min=0,
                max=10,
                value=5,
                color={"gradient":True,"ranges":{"green":[0,4],"yellow":[4,7],"red":[7,10]}},
                label='Water Quality',
                style={'margin': '20px auto'}
            )
        ], style={'width': '35%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ]),
    dcc.Interval(
        id='interval-component',
        interval=3*1000,  # Update every 3 seconds
        n_intervals=0
    )
], style={'backgroundColor': '#2F2F2F', 'padding': '20px'})

# Define callback to update graph, water quality status, and gauge
@app.callback(
    [Output('sensor-graph', 'figure'),
     Output('water-quality', 'children'),
     Output('water-quality-gauge', 'value'),
     Output('water-quality', 'style')],
    [Input('interval-component', 'n_intervals')]
)
def update_graph_live(n):
    global sensor_data

    # Read new sensor data
    new_time, new_value = read_sensor_data()
    
    if new_time and new_value:
        new_row = pd.DataFrame({'Time': [new_time], 'Sensor Value': [new_value]})
        sensor_data = pd.concat([sensor_data, new_row], ignore_index=True)

    # Determine water quality based on the last sensor value
    latest_value = sensor_data['Sensor Value'].iloc[-1] if not sensor_data.empty else 0
    if latest_value > 1:  # Example threshold value
        water_quality = "Water Quality: Bad"
        gauge_value = 8
        color = '#FF4136'
    else:
        water_quality = "Water Quality: Good"
        gauge_value = 2
        color = '#2ECC40'

    figure = {
        'data': [
            go.Scatter(
                x=sensor_data['Time'],
                y=sensor_data['Sensor Value'],
                mode='lines+markers',
                marker=dict(color='#00CC96')
            )
        ],
        'layout': go.Layout(
            title='Real-time Sensor Data',
            xaxis={'title': 'Time', 'gridcolor': '#4D4D4D'},
            yaxis={'title': 'Sensor Value', 'gridcolor': '#4D4D4D'},
            paper_bgcolor='#2F2F2F',
            plot_bgcolor='#2F2F2F',
            font={'color': '#FFFFFF'}
        )
    }
    text_style = {'fontSize': 24, 'fontWeight': 'bold', 'color': '#FFFFFF', 'backgroundColor': color, 'padding': '10px', 'margin': '20px', 'borderRadius': '5px', 'textAlign': 'center'}
    return figure, water_quality, gauge_value, text_style

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True, host='192.168.1.66', port=8050)  # Accessible from any IP
